In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import os
import gc ##garbage collection

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from tqdm import tqdm #for seeing progress

import sklearn
import sklearn.svm
import sklearn.preprocessing
import datetime

import matplotlib.pyplot as plt

import seaborn as sns

import torch #pytorch
import torch.nn as nn ##neural net stuff
import torch.nn.functional as F #useful torch function (eg.: activations)
from torch.utils.data import TensorDataset, DataLoader, Subset #for easy data batches
import torch.optim as optim #optimisation functions

from sklearn.model_selection import train_test_split #for creating test and train data
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler #scaling the data to be in range [0,1]

In [ ]:
#https://lightgbm.readthedocs.io/en/latest/
#https://www.kaggle.com/vinnsvinay/introduction-to-boosting-using-lgbm-lb-0-68357#Reading-data
#https://medium.com/@pushkarmandot/https-medium-com-pushkarmandot-what-is-lightgbm-how-to-implement-it-how-to-fine-tune-the-parameters-60347819b7fc

import lightgbm as lgb #light gbm -> creates trees

In [ ]:
df = pd.read_csv("../input/tabular-playground-series-jan-2021/train.csv")
df.head()

In [ ]:
features = df.iloc[:,1:15]
features.head()

In [ ]:
target = df.iloc[:,15]
target.head()

# Plots

In [ ]:
#since we have 14 continuous features we will graph them against the target
nrow, ncol = (7,2)
num_features = 14
fig,ax = plt.subplots(nrow,ncol,figsize=(28,56)) #3 rows 5 columns

def plot_all_features(dataframe,nrow, ncol, num, y):
    #we want to fill it up row by row
    text = "cont"
    for i in range(num):
        tx = text+str(i+1)
        
        #so first we want to find which row it is in
        row = i//ncol
        col = i-(ncol*row)
        
        ax[row,col].scatter(dataframe[tx], dataframe[y])
        ax[row,col].set_ylabel(y)
        ax[row,col].set_xlabel(tx)


plot_all_features(df,nrow,ncol,num_features,"target")

In [ ]:
##now we plot the distribution of these features with histograms
def plot_all_features_dist(dataframe,nrow,ncol, nbins=50):
    text = "cont"
    row=0
    col=0
    for i in dataframe.columns:
        #tx = text+str(i+1)
        #row = i//ncol
        #col = i-(ncol*row)
        
        ax[row,col].hist(dataframe[i], nbins, density=True)
        #ax[row,col].set_ylabel(y)
        ax[row,col].set_xlabel(str(i))
        col += 1
        if col >= ncol:
            row += 1
            col = 0

In [ ]:
def plot_all_features_dist(dataframe, start, end, nrow, ncol, nbins=50):
    text = "cont"
    row=0
    col=0
    for i in dataframe.columns[start:end]:
        #tx = text+str(i+1)
        #row = i//ncol
        #col = i-(ncol*row)
        
        ax[row,col].hist(dataframe[i], nbins, density=True)
        #ax[row,col].set_ylabel(y)
        ax[row,col].set_xlabel(str(i))
        col += 1
        if col >= ncol:
            row += 1
            col = 0

In [ ]:
fig,ax = fig,ax = plt.subplots(8,2,figsize=(28,56)) #3 rows 5 columns
plot_all_features_dist(df, 1, 16, 7, 2)

In [ ]:
#we normalise the data

def normalize(df, start, end):
    for i in df.columns[start:end]:
        df[i] = (df[i]-np.mean(df[i]))/np.std(df[i])
    return df

In [ ]:
df_normed = normalize(df, 1, 15)
fig,ax = fig,ax = plt.subplots(7,2,figsize=(28,56)) #3 rows 5 columns
plot_all_features_dist(df_normed, 1, 15, 8, 2)

In [ ]:
print("mean of target ", np.mean(df[df.columns[15]]))
print("std of target ", np.std(df[df.columns[15]]))
print("number of entries ", len(df))

# Model Creation

In [ ]:
##pytorch test
centers = 2
x1 = 5*torch.rand(3,3)
print(x1.size())
print(x1)
x1 = x1.unsqueeze(1)
print(x1.size())
print(x1)
x1 = x1.expand(-1, centers, -1)
print(x1.size())
print(x1)

c1 = torch.ones(centers,3)
print(c1)

print(x1-c1)

## Radial Basis Network

https://www.programmersought.com/article/18665200813/
https://mccormickml.com/2013/08/15/radial-basis-function-network-rbfn-tutorial/
https://www.cc.gatech.edu/~isbell/tutorials/rbf-intro.pdf

In [ ]:
def gaussian(x):
    return torch.exp(-x.pow(2))

def linear(x):
    return x

def quadratic(x):
    return x.pow(2)

def multi_quadratic(x):
    return (1+x.pow(2)).pow(0.5)

def inv_quadratic(x):
    return (1+quadratic(x)).pow(-1)

def inv_multi_quad(x):
    return multi_quadratic(x).pow(-1)

In [ ]:
class RBFN(nn.Module):
    
    
    def __init__(self, n_centers, in_features, out_features=1, basis_func=None):
        super(RBFN, self).__init__()
        #D = n_in
        #N = n_centers
        
        self.out_features = out_features
        self.in_features = in_features
        
        self.num_centers = n_centers #number of centers
        
        self.centers = nn.Parameter(torch.Tensor(self.num_centers, self.in_features), requires_grad = True) #we create N number of random centers, NxD
        self.sigmas = nn.Parameter(torch.Tensor(self.num_centers), requires_grad=True) #(1, num_centers)
        
        self.linear = nn.Linear(self.num_centers, self.out_features, bias = True) #(num_centers, out_features)
        self.reset_parameters()
        
        if basis_func != None:
            self.basis_func = basis_func
        else:
            def rbf(x):
                return torch.exp(-x.pow(2))
                
            self.basis_func = rbf
        
    def reset_parameters(self):
        nn.init.normal_(self.centers, 0, 1)
        nn.init.constant_(self.sigmas, 1)
    
        
    def kernel(self, x):
        #x is the input batch
        #M = batch_size
        #n = x.size(0) #the batch size
        #x is size (M,in_features)
        #x.unsqueeze(1) => (M, 1, in_features)
        #x.expand(-1, self.num_centers, -1) => (M, num_centers, in_features)
        x = x.unsqueeze(1).expand(-1, self.num_centers, -1)
        c = self.centers.unsqueeze(0) #(1, num_centers, in_features)
        #d (M, num_centers, in_features) => (M, num_centers)
        d = (x-c).pow(2).sum(-1).pow(0.5) #distance calculation
        d *= self.sigmas.unsqueeze(0) #(M, num_centers)
        
        return self.basis_func(d)  
    
        #1. calculate ||x - centers||
        
        #2. calculate B||x-centers||
        
        #3. return exp(B||x-centers||)
        
    def forward(self, x):
        """
        Shapes:
            x (input) has size (M, in_features) where M is an arbitrary batch size
            
            output has size (M, out_features)
            
            centers has size (N, in_features) where N is an arbitrary number of centers
            
            want to get kernel matrix of size (M,N) where K(i,j) = x_i - c_j
        
        """
        
        x = self.kernel(x)
        return self.linear(x)
        

# Dataset / Training

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
X_train, X_valid, Y_train, Y_valid = train_test_split(df_normed.iloc[:, 1:15].to_numpy(),
                                                      df_normed.iloc[:,15].to_numpy(),
                                                     test_size = 0.2,
                                                     random_state = 42)

train_set = TensorDataset(torch.from_numpy(X_train).type(torch.FloatTensor).to(device),
                         torch.from_numpy(Y_train).type(torch.FloatTensor).to(device))

valid_set = TensorDataset(torch.from_numpy(X_valid).type(torch.FloatTensor).to(device),
                         torch.from_numpy(Y_valid).type(torch.FloatTensor).to(device))

In [ ]:
batch_size = 512

train_loader = DataLoader(train_set, batch_size, pin_memory = False)
valid_loader = DataLoader(valid_set, batch_size, pin_memory = False)

In [ ]:
def fit(model, dl_train, dl_val, epochs = 50, test_epochs = 1):
    
    losses = []
    valid_losses = []
    for epoch in range(epochs):
    
        sum_loss = 0
        n = 0
        for x,y in tqdm(dl_train):
            optimizer.zero_grad()
            
            pred_y = model(x)
            y = y.unsqueeze(1)
            #print(pred_y.size())
            #print(y.size())
            loss = error(pred_y, y)
            
            loss.backward()
            optimizer.step()
            
            sum_loss += torch.sqrt(loss).cpu().data
            n+=1
        
        ####end of train
        losses.append(sum_loss / n) #store losses
        prt_str = f"epoch loss [{losses[-1]}]; "
        
        ####validation
        if epoch % test_epochs == 0:
            model = model.eval()
            
            sum_loss = 0
            n = 0
            for x,y in tqdm(dl_val):
                #optimizer.zero_grad()

                pred_y = model(x)
                y = y.unsqueeze(1)
                loss = torch.sqrt(error(pred_y, y))

                #loss.backward()
                #optimizer.step()

                sum_loss += loss.cpu().data
                n+=1

            valid_losses.append(sum_loss / n)
            prt_str += f"validation_loss [{valid_losses[-1]}]"
            
            model = model.train()
            
        ## we print the epoch data
        print(prt_str)
        
    return losses, valid_losses    

In [ ]:
rbf_model = RBFN(14, 14, 1, gaussian).to(device).eval() #14 centers, 14 features, 1 output
print(rbf_model)
#x1 = torch.ones(1,14)
#print(x1)
#y1 = rbf_model(x1)
#print(y1.size())
#print(y1)

In [ ]:
rbf_model = rbf_model.to(device).train()
lr = 0.001
weight_decay = 0 #1e-4
error = nn.MSELoss()
optimizer = optim.Adam(rbf_model.parameters(), lr=lr, weight_decay = weight_decay, amsgrad = True)

#losses, valid_losses = fit(rbf_model, train_loader, valid_loader, 50, 1)

In [ ]:
#plt.plot(losses[1:])
#plt.plot(valid_losses[1:])

In [ ]:
import optuna

In [ ]:
def create_model(trial):
    num_layers_centered = trial.suggest_int("num_centered_layers", 1, 3)
    num_layers_linear = trial.suggest_int("num_linear_layers", 0, 3)
    layers =[]
    radial_func = gaussian
    
    in_features = 14
    prev_centers = 14**3
    max_out = 14**3
    
    for i in range(num_layers_centered):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 2**10)
        num_centers = trial.suggest_int("n_centers_l{}".format(i), 2, 2**10)
        layers.append(RBFN(num_centers, in_features, out_features, radial_func))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
        #max_out = out_features
        #prev_centers = num_centers
    
    for i in range(num_layers_linear):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 1+14**2)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.1, 0.5)
        layers.append(nn.Dropout(p))
        in_features = out_features
    
    layers.append(nn.Linear(in_features, 1))
    return nn.Sequential(*layers)
        

In [ ]:
 ##hyperparameter optimisation

def objective(trial):
    dl_train = train_loader
    dl_val = valid_loader
    error = nn.MSELoss()
    #num_c = trial.suggest_int("num_c", 1, 14**3)
    epochs = trial.suggest_int("epochs", 1, 200)
    
    #model = RBFN(num_c,14,1, gaussian).to(device) ##fixed model
    model = create_model(trial).to(device) #variational model
    print(model)
    
    lr = trial.suggest_loguniform("lr", 1e-5, 1e-1)    
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)
    
    for epoch in tqdm(range(epochs)):
    
        for x,y in dl_train:
            optimizer.zero_grad()
            
            pred_y = model(x)
            y = y.unsqueeze(1)
            #print(pred_y.size())
            #print(y.size())
            loss = error(pred_y, y)
            
            loss.backward()
            optimizer.step()
        
        ####end of train
        
        ####validation
        valid_loss = 0
        if epoch % 1 == 0:
            model = model.eval()
            n=0
            for x,y in dl_val:
                #optimizer.zero_grad()

                pred_y = model(x)
                y = y.unsqueeze(1)
                loss = torch.sqrt(error(pred_y, y))

                #loss.backward()
                #optimizer.step()

                valid_loss += loss.cpu().data
                n+=1

            valid_loss /= n        
            model = model.train()
            
        ## we print the epoch data
        
        trial.report(valid_loss, epoch)
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()
        
    return valid_loss 

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

In [ ]:
#from sklearn.kernel_ridge import KernelRidge

In [ ]:
#regr_model = sklearn.svm.SVR(kernel="rbf", C=1.0, epsilon = 0.1)
#regr_model = KernelRidge(alpha=1.0, kernel="rbf")

In [ ]:
#regr_model.fit(df_normed.iloc[:,1:15], df_normed["target"])

In [ ]:
#test_df = pd.read_csv("../input/tabular-playground-series-jan-2021/test.csv")
#test_df = normalize(test_df, 1, 15)
#predictions = regr_model.predict(test_df.iloc[:,1:15])

In [ ]:
#sam_sub = pd.read_csv("../input/tabular-playground-series-jan-2021/sample_submission.csv")
#sam_sub["target"] = predictions

#sam_sub.to_csv("submission.csv", index=False)